### Please use PySpark kernel for this notebook

In [1]:
lines = sc.textFile("gs://abdul-pyspark/notebooks/jupyter/ml-100k/u.data")

##### Create a Pipelined/Pair RDD with (Movie ID, Weighted Rating)

In [2]:
w_rating = {5:1.25, 4:1.15, 3:1, 2:0.9, 1:0.7}
rating_bc = sc.broadcast(w_rating)

In [3]:
rating_bc.value[2]

0.9

In [4]:
movies = lines.map(lambda x: (int(x.split()[1]), int(x.split()[2])))


In [5]:
w_movies = movies.map(lambda x: (x[0], rating_bc.value[x[1]]))

In [6]:
movieCounts = w_movies.reduceByKey(lambda x, y: x + y)

type(movieCounts)

pyspark.rdd.PipelinedRDD

In [7]:
movieCounts.count()

1682

In [8]:
#movieCounts.collect()

### Loading movie names

In [9]:
def load_movie_names():
    names = {}
    # the path below doesn't work as a "local" function on the cluster master VM
    with open("/home/abdullahkhan/u.item", encoding='ISO-8859-1') as f:
        for line in f:
            tokens = line.strip().split("|")
            names[int(tokens[0])] = tokens[1]
    return names

In [10]:
names = load_movie_names()

##### Create a broadcast variable for the movie name dict

In [11]:
names_bc = sc.broadcast(names)

In [12]:
movieCountsWithNames = movieCounts.map(lambda x : (names_bc.value[x[0]], x[1]))

In [ ]:
#movieCountsWithNames.collect()

In [13]:
sortedResultsWithNames = movieCountsWithNames.sortBy(lambda x: x[1], ascending=False).collect()

In [14]:
for i in range(10):
    print(sortedResultsWithNames[i])

('Star Wars (1977)', 686.3499999999992)
('Fargo (1996)', 585.5999999999999)
('Return of the Jedi (1983)', 576.2500000000002)
('Contact (1997)', 565.9000000000004)
('English Patient, The (1996)', 524.5000000000005)
('Scream (1996)', 508.2000000000006)
('Toy Story (1995)', 506.9500000000006)
('Liar Liar (1997)', 497.75000000000045)
('Raiders of the Lost Ark (1981)', 489.3000000000006)
('Godfather, The (1972)', 482.9500000000006)
